In [36]:
import numpy as np
from itertools import combinations

In [37]:
%run std_input_matrix.ipynb


Vincolo numero 0
Inserire coefficiente numero 0
Inserire coefficiente numero 1
Inserire coefficiente numero 2
Inserire coefficiente numero 3
Inserire coefficiente numero 4
Inserire coefficiente numero 5
Inserire coefficiente numero 6
Inserire termine noto: 

Vincolo numero 1
Inserire coefficiente numero 0
Inserire coefficiente numero 1
Inserire coefficiente numero 2
Inserire coefficiente numero 3
Inserire coefficiente numero 4
Inserire coefficiente numero 5
Inserire coefficiente numero 6
Inserire termine noto: 

Vincolo numero 2
Inserire coefficiente numero 0
Inserire coefficiente numero 1
Inserire coefficiente numero 2
Inserire coefficiente numero 3
Inserire coefficiente numero 4
Inserire coefficiente numero 5
Inserire coefficiente numero 6
Inserire termine noto: 
[[ 1. -1.  0.  0.  1.  0.  0.]
 [ 1. -1. -1.  1.  0. -1.  0.]
 [ 1. -1.  1. -1.  0.  0. -1.]]
[[2.]
 [0.]
 [1.]]
[[ 1. -1.  0.  0.  1.  0.  0.]
 [ 1. -1. -1.  1.  0. -1.  0.]
 [ 1. -1.  1. -1.  0.  0. -1.]]
[[ 1. -1.  0.  0

Ogni problema di programmazione lineare può essere trasformato in un problema equivalente formulato in forma standard.

Definizione: Un problema di PL si dice formulato in forma standard se si tratta di minimizzare una funzione obiettivo lineare soggetta a vincoli di uguaglianza, se le variabili di decisione sono vincolate ad essere nonnegative e il vettore dei termini noti è nonnegativo.

Infatti, è sempre possibile assumere che la funzione obiettivo lineare z(x) sia da
minimizzare, in quanto il valore della variabile x∗ che rende massima z(x)
rende minima la stessa funzione cambiata di segno.

Un qualsiasi vincolo può essere trasformato in vincolo di uguaglianza introducendo opportune variabili aggiuntive nonnegative: 
1. nel caso di un vincolo (<=) avremo una variabile di slack (positiva, che rappresenta la differenza nonnegativa tra il secondo ed il primo
membro della disuguaglianza);
2. nel caso di un vincolo (>=) avremo una variabile di surplus (negativa, rappresenta la differenza nonnegativa tra il primo ed il secondo
membro della disuguaglianza).

Se un problema di PL formulato in forma standard ammette una soluzione ottima finita, allora
esiste una soluzione ammissibile di base ottima.

Il metodo del simplesso si basa su tale considerazione e cerca una soluzione ottima al problema
muovendosi da una soluzione ammissibile di base ad una adiacente sempre nella direzione lungo la quale si verifica un decremento del valore corrente della funzione obiettivo.

Il metodo del simplesso, passando da una soluzione ammissibile di base ad una soluzione ammissibile di base adiacente e terminando quando non esiste una soluzione ammissibile di base
migliore, può essere interpretato come algoritmo di ricerca locale. Fortunatamente, però, nel caso di problemi di PL, dal momento che si tratta di ottimizzare una funzione obiettivo convessa su una regione ammissibile convessa, si è certi che l’ottimo locale restituito in output dal metodo del simplesso è ottimo anche globalmente.

In [38]:
# 'flag' è la variabile binaria che serve a stoppare il while successivo
# una volta creata la matrice B (sub_matrix) che rappresenta la matrice di base 
flag = False
# 'indici' è una lista di tuple dove ogni tupla rappresenta una combinazione
# di possibili colonne di base
indici = list(combinations(range(matrix_A.shape[1]), rank_A))
n_comb = len(indici)
print(indici)
count = 0
while flag==False:
    # faccio una copia della matrice iniziale in modo da conservare gli indici 
    matrix_Aprime = matrix_A.copy()
    print(matrix_Aprime)
    # creo la matrice dei vettori che scarto (linearmente dipendenti)
    del_vect = np.array([])

    # matrice di base B
    sub_matrix = []
    for r in range(rank_A):
        sub_matrix.append(matrix_Aprime[:, indici[0][r]]) 

    rank_subA = np.linalg.matrix_rank(sub_matrix)
    if rank_subA != rank_A:
        print('rank_subA: ', rank_subA)
        print('rank_A: ', rank_A)
    
    # vettore indici variabili di base
    vettori_base = [indici[0][r] for r in range(rank_A)]
    print('vettori_base: ', vettori_base)
    # vettore indici delle variabili fuori base
    vettori_fuori_base = [i for i in range(matrix_A.shape[1]) if i not in vettori_base]

    # nel caso in cui la sottomatrice selezionata 
    # abbia rango minore della matrice A
    while rank_subA != rank_A:
        indici.remove(indici[0])
        sub_matrix = []
        for r in range(rank_A):
            sub_matrix.append(matrix_Aprime[:, indici[0][r]])  
        rank_subA = np.linalg.matrix_rank(sub_matrix) 

        # vogliamo sapere gli indici relativi alle colonne indipendenti
        lambdas =  np.real(np.linalg.eigvals(sub_matrix))
        lambdas = np.array([round(i, 6) for i in lambdas])
        print('LambdaS:\n', lambdas)
        vettori_dipendenti = np.where(lambdas==0) # restituisce gli indici dei vettori linearmente dipendenti
        vettori_independenti = np.where(lambdas!=0) # restituisce gli indici dei vettori linearmente indipendenti
        print('vettori dipendenti ', len(vettori_dipendenti))
        if len(vettori_dipendenti) != 0:
            # where the magic happendS
            del_vect = np.append(del_vect, matrix_Aprime[vettori_dipendenti])
            matrix_Aprime = np.delete(matrix_Aprime, vettori_dipendenti, axis=1)
        # devo lavarmi
        print('rank_subA: ', rank_subA)
        print('rank_A: ', rank_A)
        vettori_base = vettori_independenti[:rank_A][0]
        print('vettori_base_2: ', vettori_base)
        vettori_fuori_base = [i for i in range(matrix_A.shape[1]) if i not in vettori_base]
    print('vettori base:\n', vettori_base)
    print('vettori fuori base:\n', vettori_fuori_base)

    # x = B^(-1)*b
    # dove B è la matrice di colonne linearmente indipendenti (submatrix)
    # trasponiamo submatrix
    sub_matrix = np.array(sub_matrix).T
    # invertiamo submatrix
    print(sub_matrix)
    inv_subm = np.linalg.inv(sub_matrix)
    # calcoliamo la soluzione con il prodotto righe per colonna
    xb = np.dot(inv_subm, termini_noti)
    print('Soluzione:\n', xb)
    # a questo punto, aggiorniamo la variabile 'flag' controllando che i coefficienti di x siano tutti positivi
    flag = (xb>0).all() # restituisce 'True' se la condizione è soddisfatta da tutti i coeff
    # condizione di stop per il while:
    # si stoppa quando cicla un numero pari 
    # al numero di combinazioni possibili
    indici.remove(indici[0])
    count = count + 1
    if count == n_comb:
        print('Non esiste una soluzione ottimale finita.')
        break

[(0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 1, 6), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 2, 6), (0, 3, 4), (0, 3, 5), (0, 3, 6), (0, 4, 5), (0, 4, 6), (0, 5, 6), (1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 2, 6), (1, 3, 4), (1, 3, 5), (1, 3, 6), (1, 4, 5), (1, 4, 6), (1, 5, 6), (2, 3, 4), (2, 3, 5), (2, 3, 6), (2, 4, 5), (2, 4, 6), (2, 5, 6), (3, 4, 5), (3, 4, 6), (3, 5, 6), (4, 5, 6)]
[[ 1. -1.  0.  0.  1.  0.  0.]
 [ 1. -1. -1.  1.  0. -1.  0.]
 [ 1. -1.  1. -1.  0.  0. -1.]]
rank_subA:  2
rank_A:  3
vettori_base:  [0, 1, 2]
LambdaS:
 [ 0.  -0.5 -0.5]
vettori dipendenti  1
rank_subA:  2
rank_A:  3
vettori_base_2:  [1 2]
LambdaS:
 [-1.  -0.5 -0.5]
vettori dipendenti  1
rank_subA:  3
rank_A:  3
vettori_base_2:  [0 1 2]
vettori base:
 [0 1 2]
vettori fuori base:
 [3, 4, 5, 6]
[[-1.  0.  0.]
 [-1. -1. -1.]
 [-1.  1.  0.]]
Soluzione:
 [[-2.]
 [-1.]
 [ 3.]]
[[ 1. -1.  0.  0.  1.  0.  0.]
 [ 1. -1. -1.  1.  0. -1.  0.]
 [ 1. -1.  1. -1.  0.  0. -1.]]
rank_subA:  2
rank_A:  3
vettori_base:  [0,

In [39]:
print(vettori_base)

[0, 2, 4]


Innanzitutto, muovendoci da $x$ a $x + \theta d$ noi vogliamo comunque rimanere all’interno della regione ammissibile $P$, cioè vogliamo preservare l’ammissibilità della nuova soluzione $x + \theta d$. 
Formalmente, ciò equivale a richiedere che:
$$A(x+ \theta d) = b$$
$$ 0 = Ad = \sum_{i=1}^{m} A_{B(i)} d_{B(i)} + \sum_{l \neq B(1),...,B(m)} A_{l}d_{l} $$
Ricordando che, per ogni $i \neq j$:
$$d_{j}=1, d_{i}=0$$
$$ 0 = Ad = Bd_{B} + A_{j} $$
Dunque:
$$ d_{B} = -B^{-1}A_{j} $$
Supponiamo di aver individuato una direzione ammissibile d e di essere pronti a spostarci da $x$ al punto $x + \theta d$. Effettueremo lo spostamento solo qualora esso fosse conveniente in termini di decremento del valore della funzione obiettivo, che ora sarà:
$$ c'(x+\theta d) = c'x + \theta c'd$$
$$ c'd = c'_{B}d_{B} + c_{j} = c_{j} - c'_{B}B^{-1}A_{j} $$
per essere sicuri che la corrente soluzione ammissibile di base non degenere sia ottima, è sufficiente controllare che tutti i costi ridotti delle variabili correntemente non in base siano non negativi.

Dunque, abbiamo raggiunto il nostro scopo: passare da una soluzione ammissibile di base (che è un vertice del poliedro descritto dall’insieme dei vincoli) ad una adiacente (vertice adiacente) cui corrisponde un valore della funzione obiettivo inferiore.

Definiamo:
$$ u = - d_{B} = B^{-1} A_{j} $$
Se $u_{i} \leq 0$, per ogni $i = 1,2,...,m$ allora $\theta* = \infty$, il valore ottimo della funzione obiettivo è $-\infty$ e l’algoritmo termina.

Altrimenti, si calcola:
$$ \theta * = min_{\{i=1,...,m | u_{i}>0\}} \{\frac{x_{i}}{u_{i}}\}\$$
Sia $l$ l’indice della variabile di base per cui $\theta* = \frac{x_B{l}}{u_{l}}$.

Si formi una nuova matrice di base rimpiazzando la colonna $A_B{l}$ con la colonna $A_{j}$ e si calcoli la nuova soluzione ammissibile di base $\bar{x}$ data da:
$$ \bar{x} = 0 $$
$$ \bar{x_{j}} = \theta* $$
$$ \bar{x_{B(i)}} = x_{B(i)} - \theta*u_{i} $$
$$ \text{con $i \neq l$} $$



In [40]:
list_coff

[1, -1, 0, 0, 0, 0, 0]

In [41]:
# cprimeb = [list_coff[i] for i in vettori_base]
for i in vettori_base:
    print('indice: ', i)
    print('coefficiente: ',list_coff[i])

indice:  0
coefficiente:  1
indice:  2
coefficiente:  0
indice:  4
coefficiente:  0


In [42]:
flag_2 = False
while flag_2==False:
    # lista dei coefficienti dei vettori base
    cprimeb = [list_coff[i] for i in vettori_base]
    print(cprimeb)
    # inizializzo lista di tutti i costi ridotti
    c_j_list = [] 
    # inizializzo lista in cui salviamo l'indice delle variabili relative ad un costo ridotto negativo
    neg_c_j_ind = [] 
    for j in vettori_fuori_base:
        # calcolo dei costi:
        # c_j = cj - c'b * B(-1) * Aj
        c_j = list_coff[j] - np.dot(np.dot(cprimeb, np.linalg.inv(sub_matrix)), matrix_A[:,j])
        c_j_list.append(c_j)
        if c_j<0:
            neg_c_j_ind.append(j)
    # vettore dei costi
    c_j_list = np.array(c_j_list)
    print('c_j:\n', c_j_list)
    print('neg_cj:\n', neg_c_j_ind)
    if (c_j_list>=0).all():
        # se tutti i costi ridotti sono >= 0, la soluzione ottima è stata trovata:
        print('SOLUZIONE OTTIMA:\n', xb)
        print("variabili in base",vettori_base)
        flag_2 = True
    else:
        # scegliamo l'indice più piccolo tra le variabili che hanno costo ridotto negativo
        min_ind = np.min(neg_c_j_ind) # REGOLA DI BLAND
        # calcolo u = B(-1) * A(indice minimo scelto)
        u = np.dot(np.linalg.inv(sub_matrix), matrix_A[:,min_ind])
        ind_u_i = [i for i in range(len(u)) if u[i]>0]
        if (u<=0).all():
            # se tutti i coefficienti di u sono <= 0, soluzione -> -inf 
            print('Il valore ottimo della funzione obiettivo è meno infinito.')
        else:
            # altrimenti, calcoliamo theta minimo
            # theta = min {x_i/u_i} 
            # per tutti gli i | u_i > 0
            # STELLINA
            theta = np.min([k/w for k, w in zip(xb, u) if w>0])
            # una volta trovato il theta, siamo interessati all'indice corrispondente
            # alla variabile in base a cui corrisponde il theta min
            ind_out_var = [k/w for k, w in zip(xb, u) if w>0].index(theta)
            # indice corrispondente alla variabile di base da far uscire
            out_var = vettori_base[ind_out_var]
            # sostituiamo la colonna corrispondente all'indice della variabile uscente
            # con la colonna relativa alla variabile corrispondente all'indice scelto
            # con la regola di Bland -> essa sarà una nuova colonna di base
            print('sub_matrix:\n', sub_matrix)
            # cambiamo indice delle variabili di base
            vettori_base[ind_out_var] = min_ind
            # ridefiniamo la matrice di base
            sub_matrix = matrix_A[:, vettori_base]
            print('sub_matrix:\n', sub_matrix)
            new_xb_i = [x_i-theta*u_i for x_i, u_i in zip(xb,u)]
            print(new_xb_i)
            xb=np.zeros(rank_A)
            for i in range(len(xb)):
                xb[i]=new_xb_i[i]  
                if i==ind_out_var:
                    xb[i]=theta
            print("soluzione:\n", xb)  
            print("variabili di base:\n",vettori_base)
        

[1, 0, 0]
c_j:
 [0.  0.  0.5 0.5]
neg_cj:
 []
SOLUZIONE OTTIMA:
 [[0.5]
 [0.5]
 [1.5]]
variabili in base [0, 2, 4]


In [43]:
#calcolo costo ottimo della funzione obiettivo
list_coff_base=[list_coff[i] for i in vettori_base]
costo=np.dot(list_coff_base,xb)
costo

array([0.5])